### 東京都文京区物件スクレイピング

参考文献
スクレイピング動画
- https://www.youtube.com/watch?v=l9KRgSUeNXk
- https://www.youtube.com/watch?v=gixuk9qFpRg
- https://www.youtube.com/watch?v=_H9bdFOZ4vs
- https://www.youtube.com/watch?v=Lq4xHWnT_KE

スプレッドシート動画
- https://youtu.be/fFSGPciIkfI?si=MQ4wrvIiFpmwgdCR

In [18]:
#ライブラリのインポート
from bs4 import BeautifulSoup
import re #不要文字削除
import requests
from time import sleep
from tqdm import tqdm   #for文の進捗確認
import pandas as pd

In [19]:
# 最後のページの数値を取得
url = 'https://suumo.jp/chintai/tokyo/sc_bunkyo/?page={}'
res = requests.get(url.format(1))
#1ページ目のページネーションに全ページの番号が表示される
#1ページ目のページネーション部分をスクレイピングし、その中の最後の要素（最後のページ番号）を取得
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text, 'html.parser')
last_page = int(soup.find('ol', class_='pagination-parts').find_all('li')[-1].text)
#[-1]でページネーションのリストから最後の要素を取得
#textでページ番号を取得
#intで整数型に変換

In [20]:
#空のリストを作成
data_list = []

In [21]:
# 正常にHTML情報が取得できれば以下のコードを実行
if res.status_code == 200:

    #文京区のSUUMO掲載物件全ページ情報を取得
    for page in tqdm(range(1, last_page +1)): #for文の進捗を確認
    #range(start, stop) 関数は、startから始まってstop-1までの数値のシーケンスを生成
    #stopの値自体はシーケンスに含まれないため +1で最後のページもスクレイピング
        target_url = url.format(page)
        #ページ取得できているかの確認
        print("data_listの大きさ:",len(data_list))
        print(target_url)

        #requestを使ってURLにアクセス
        res = requests.get(target_url)
        #相手サイトの負荷軽減
        sleep(1)
        #文字化け防止
        res.encoding = 'utf-8'
        #取得したHTMLをBeautifulSoupで解析
        soup = BeautifulSoup(res.text, 'html.parser')

        #全ての物件情報取得
        contents = soup.find_all('div', class_= 'cassetteitem')

        #for文で物件・部屋情報取得
        for content in contents:
            #物件・部屋情報を解析
            detail = content.find('div', class_='cassetteitem-detail')
            table = content.find('table', class_='cassetteitem_other')

            #物件情報から必要情報を取得
            title = detail.find('div', class_='cassetteitem_content-title').text
            address = detail.find('li', class_='cassetteitem_detail-col1').text
            access = detail.find('li', class_='cassetteitem_detail-col2').text
            age = detail.find('li', class_='cassetteitem_detail-col3').text

            #部屋情報を取得
            tr_tags = table.find_all('tr', class_='js-cassette_link')

            #for文で部屋情報取得
            for tr_tag in tr_tags:
                #部屋情報から必要情報を取得
                floor, price, first_fee, capacity = tr_tag.find_all('td')[2:6]
                #さらに細かい情報取得
                rent, administration = price.find_all('li')
                deposit, gratuity = first_fee.find_all('li')
                madori, menseki = capacity.find_all('li')
                #取得した全ての情報を辞書に格納
                data = {
                    'title' : title,
                    'address' : address,
                    'access' : access,
                    'age' : age,
                    'floor': floor.text,
                    'rent' : rent.text,
                    'administration' : administration.text,
                    'deposit' : deposit.text,
                    'gratuity' : gratuity.text,
                    'madori' : madori.text,
                    'menseki' : menseki.text
                }
                #取得した辞書を格納
                data_list.append(data)

  0%|          | 0/171 [00:00<?, ?it/s]

data_listの大きさ: 0
https://suumo.jp/chintai/tokyo/sc_bunkyo/?page=1


  1%|          | 1/171 [00:02<07:31,  2.65s/it]

data_listの大きさ: 106
https://suumo.jp/chintai/tokyo/sc_bunkyo/?page=2


  1%|          | 2/171 [00:05<07:12,  2.56s/it]

...

 99%|█████████▉| 169/171 [05:41<00:03,  1.88s/it]

data_listの大きさ: 6639
https://suumo.jp/chintai/tokyo/sc_bunkyo/?page=170


 99%|█████████▉| 170/171 [05:43<00:01,  1.88s/it]

data_listの大きさ: 6667
https://suumo.jp/chintai/tokyo/sc_bunkyo/?page=171


100%|██████████| 171/171 [05:45<00:00,  2.02s/it]


utf-8, utf-8-sig の違い
https://qiita.com/showmurai/items/60d32006d13512ffeaff

エンコードのコーデックに 'utf-8-sig' とすると、BOM有りの場合スキップして読み込んでくれる。
BOM無しの場合はそのままUTF-8として読み込むことが可能。

UTF-8 には BOM(Byte order mark)が付く場合がある。
UTF-8にBOMを付けるのは、Windows'メモ帳'やExcel。LinuxやMacは基本的にBOM無しUTF-8で扱う。

In [22]:
#最後のインデックスを確認
data_list[-1]

{'title': 'ＪＲ山手線 田端駅 2階建 築63年',
 'address': '東京都文京区本駒込４',
 'access': '\nＪＲ山手線/田端駅 歩13分\n都営三田線/千石駅 歩17分\n東京メトロ南北線/本駒込駅 歩18分\n',
 'age': '\n築63年\n2階建\n',
 'floor': '\r\n\t\t\t\t\t\t\t\t\t\t\t2階',
 'rent': '3万円',
 'administration': '2000円',
 'deposit': '-',
 'gratuity': '-',
 'madori': 'ワンルーム',
 'menseki': '8.2m2'}

In [23]:
#データフレームを作成
df = pd.DataFrame(data_list)
df2 = df.applymap(lambda x: re.sub('\n', ' ', x)) #`\n`を半角スペースに置換
df3 = df2.applymap(lambda x: re.sub('\r', ' ', x)) #`\r`を半角スペースに置換
df_Final = df3.applymap(lambda x: re.sub('\t', ' ', x)) #`\t`を半角スペースに置換
df_Final.to_csv('SUUMO_bunkyo.csv', index=False, encoding='utf-8-sig')

print("csv出力が完了しました")

#df.applymap() : データフレーム全体に対して、カッコ内の関数を適応
#lambda x: re.sub('\n', ' ', x)) : 引数で渡された`x`に対して、`\n`を半角スペースに置換

df_Final.head()

csv出力が完了しました


,title,address,access,age,floor,rent,administration,deposit,gratuity,madori,menseki
0,アーバンステージ千石II,東京都文京区千石４,都営三田線/千石駅 歩6分 ＪＲ山手線/巣鴨駅 歩9分 東京メトロ丸ノ内線/新大塚駅 歩12分,築6年 4階建,1階,10.8万円,8000円,10.8万円,10.8万円,1LDK,30.06m2
1,アーバンステージ千石II,東京都文京区千石４,都営三田線/千石駅 歩6分 ＪＲ山手線/巣鴨駅 歩9分 東京メトロ丸ノ内線/新大塚駅 歩12分,築6年 4階建,2階,15.7万円,10000円,15.7万円,15.7万円,1LDK,41.33m2
2,ＡＴＬＡＳ Ｃａｒｏ 東大前,東京都文京区弥生２,東京メトロ千代田線/根津駅 歩6分 東京メトロ南北線/東大前駅 歩10分 東京メトロ千代田...,築5年 5階建,3階,9万円,6500円,-,9万円,ワンルーム,20.01m2
3,ＡＴＬＡＳ Ｃａｒｏ 東大前,東京都文京区弥生２,東京メトロ千代田線/根津駅 歩6分 東京メトロ南北線/東大前駅 歩10分 東京メトロ千代田...,築5年 5階建,5階,15万円,6500円,-,15万円,1LDK,40.27m2
4,モンテ駒込,東京都文京区千駄木５,東京メトロ南北線/本駒込駅 歩8分 東京メトロ千代田線/千駄木駅 歩13分 都営三田線/白...,築2年 5階建,4階,10.3万円,10000円,-,-,1LDK,25.06m2


In [24]:
#dfの大きさ確認
df.shape

(6672, 11)

In [25]:
#物件名の重複を削除した場合の、dfの大きさを確認 = 物件数
len(df.title.unique())

3224

スクレイピングエラーが発生する場合考えられる事項。

以下、調べた結果
- ウェブサイトのアンチスクレイピング機能
    - 多くのウェブサイトは、スクレイピングを検出してブロックするための機能を備えている。頻繁なアクセスや一定のパターンのアクセスが検出されると、IPアドレスが一時的にブロックされることがある。

- リクエストの制限
    - サーバーは一定時間内のリクエスト数に制限を設けている場合がある。これに達すると、それ以上のリクエストは拒否されるか、エラーが返される。

- タイムアウトエラー
    - ネットワークの問題やサーバーの過負荷などにより、リクエストがタイムアウトすることがある。

- ページ構造の変更
    - スクレイピングしているウェブサイトがページの構造を変更した場合、コードが正しく機能しなくなることがある。

- スクリプトのエラー
    - スクリプト自体に問題がある場合。例えば、特定の条件下でエラーが発生する可能性があるコードなど。

一般的には、ウェブサイトやサーバーのポリシーによって、一度のセッションで取得できるデータ量やリクエスト数に制限が設けられることがある。

### スプレッドシートに出力

In [26]:
#google spread sheets 出力
#ライブラリのインポート
import gspread
from oauth2client.service_account import ServiceAccountCredentials

#環境変数関連
from dotenv import load_dotenv
load_dotenv()
import os

In [27]:
#スコープとjsonファイルを使って認証情報を取得
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = os.getenv('SERVICE_ACCOUNT_FILE')
credentials = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_FILE, SCOPES)

In [28]:
#認証情報をauthorize関数に渡してスプレッドシートの操作権を取得
gs = gspread.authorize(credentials)

#シート情報を取得して変数に代入
SPREADSHEET_KEY = os.getenv('SPREADSHEET_KEY')
workbook = gs.open_by_key(SPREADSHEET_KEY)
worksheet = workbook.worksheet("suumo_DB")

In [29]:
#dfから値を習得
values = [df.columns.values.tolist()] + df.values.tolist()

# ワークシートの指定したセル(B2)から値を追加
worksheet.update("B2", values)

/var/folders/3d/lpwdp4ts3r5grm10hjnwtbnm0000gn/T/ipykernel_38019/2641480696.py:5: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update("B2", values)


{'spreadsheetId': '14JE9_LppdTkFaE533XHqdfOmKSLTas24s_j8RGP_u8A',
 'updatedRange': 'suumo_DB!B2:L6674',
 'updatedRows': 6673,
 'updatedColumns': 11,
 'updatedCells': 73403}